In [111]:
import pandas as pd
import numpy as np
import random
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
video_space = pd.read_csv("video_space.csv")
video_space = video_space.set_index('video_id')
patient_features = pd.read_csv("patient_features.csv")
watch_events = pd.read_csv("video_watched.csv")

In [12]:
del patient_features['Unnamed: 0']

In [13]:
#patient_features

In [102]:
#video_space

In [14]:
#watch_events

In [103]:
cos_sim = cosine_similarity(video_space)
cos_sim = pd.DataFrame(cos_sim, index = video_space.index, columns = video_space.index)
cos_sim

video_id,624,620,787,771,2587,707,2595,3021,3023,3020,...,5747,5746,5286,6461,6462,6464,6463,6449,6452,6450
video_id,,,,,,,,,,,,,,,,,,,,,
624,1.000000,0.055253,0.190571,0.184338,0.067437,0.136357,0.098136,0.201664,0.002268,0.008633,...,0.166290,0.135751,0.135379,0.146237,0.121481,0.178764,0.144975,0.146237,0.178764,0.121481
620,0.055253,1.000000,0.016997,0.008848,0.010294,0.000960,0.007204,0.138950,0.084866,0.127832,...,0.100915,0.027810,0.029183,0.001039,0.027836,0.027353,0.001176,0.001039,0.027353,0.027836
787,0.190571,0.016997,1.000000,0.116471,0.129313,0.181760,0.104780,0.152628,0.023758,0.053694,...,0.140925,0.133654,0.134100,0.183719,0.162406,0.385765,0.231629,0.183719,0.385765,0.162406
771,0.184338,0.008848,0.116471,1.000000,0.133547,0.166441,0.153672,0.157543,0.001854,0.007994,...,0.239669,0.137267,0.138147,0.148484,0.134962,0.130890,0.128451,0.148484,0.130890,0.134962
2587,0.067437,0.010294,0.129313,0.133547,1.000000,0.128171,0.380552,0.120632,0.061247,0.119014,...,0.066117,0.121906,0.117258,0.326194,0.183022,0.067888,0.212436,0.326194,0.067888,0.183022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6464,0.178764,0.027353,0.385765,0.130890,0.067888,0.169592,0.049308,0.138149,0.001466,0.008713,...,0.145211,0.072767,0.076343,0.116137,0.099086,1.000000,0.152224,0.116137,1.000000,0.099086
6463,0.144975,0.001176,0.231629,0.128451,0.212436,0.112220,0.119670,0.151403,0.002455,0.020796,...,0.133446,0.168748,0.174653,0.186251,0.231349,0.152224,1.000000,0.186251,0.152224,0.231349
6449,0.146237,0.001039,0.183719,0.148484,0.326194,0.129541,0.249280,0.112687,0.002169,0.039181,...,0.107166,0.109936,0.109591,1.000000,0.203358,0.116137,0.186251,1.000000,0.116137,0.203358


In [104]:
# build watch list for each patient
watch_list = {}

for ind in watch_events.index:
    
    patient_id = watch_events.at[ind, 'patient_id']
    video_id = watch_events.at[ind, 'video_id']
    
    if patient_id in watch_list.keys():
        temp_list = watch_list.get(patient_id)
        if video_id not in temp_list:
            temp_list.append(video_id)
            watch_list[patient_id] = temp_list
    else:
        temp_list = []
        temp_list.append(video_id)
        watch_list[patient_id] = temp_list
        
    

In [107]:
# to test, get patients with more than 10 watch events
test_watched_list = {}

# for each patient
for patient in watch_list.keys():
    
    watched = watch_list.get(patient)

    # if list of watched videos less than 3, don't use in test (cold start concerns)
    if(len(watched) > 6):
        test_watched_list[patient] = watched
    

# Run from here

In [108]:
# pull two videos from each patient to try and guess with recommender
df = pd.DataFrame(index = test_watched_list.keys(), columns = ['watch_list', 'test_in', 'test_out', 'recommendation'])

for patient in test_watched_list.keys():
    
    patient_watched_list = test_watched_list.get(patient)
    df.at[patient, 'watch_list'] = patient_watched_list
    
    test_out = []
    test_in = []
    
    for i in range(0,2):
        rand = random.randint(0, len(patient_watched_list) - 1)
        test_out.append(patient_watched_list[rand])
        
    for el in patient_watched_list:
        if el not in test_out:
            test_in.append(el)
    
    df.at[patient, 'test_in'] = test_in
    df.at[patient, 'test_out'] = test_out


In [109]:
df_ = df
TOP_SIMILAR = 2
REC_LIST_SIZE = 10

# try to predcit for 10 patients
for i in range(0, 3000):
    
    # get random patient
    rand = random.randint(0, len(df.index))
    patient = df_.index[rand]
    
    #get their watched list for this test
    watched = df_.at[patient, 'test_in']
    #vars needed for bookkeeping
    rec_list = []
    
    # for each video in the patients watched list
    for video in watched:

        #get cos_sim series
        sim = cos_sim[video]
        sim = sim.sort_values(ascending = False)
        sim = pd.DataFrame(sim).reset_index()
        sim = sim.where(sim[video] < 1).dropna()
        
        start_index = sim.index[0]

        # for each video in that cluster
        for i in range(start_index, start_index + TOP_SIMILAR):
            rec_video = sim.at[i, 'video_id']
            if rec_video not in df_.at[patient, 'test_in']:
                #if list isn't too big
                if len(rec_list) < REC_LIST_SIZE:
                    #if video wasn't already added
                    if rec_video not in rec_list:
                        #rec_category = video_to_cat.at[rec_video, 'category']
                        #if rec_category is primary:
                        rec_list.append(int(rec_video))
            # endif
        # end for
    #end for
        
    df_.at[patient, 'recommendation'] = rec_list
    
#end for


df_ = df_.dropna()

good = 0
total = 0

for patient in df_.index:
    
    recs = df_.at[patient, 'recommendation']
    
    test_out = df_.at[patient, 'test_out']
    
    for video in test_out:
        total = total + 1
        if video in recs:
            good = good + 1
        #end if
    #end for
    
    
#end for

print("Good is " + str(good))
print("Total is "+ str(total))

Good is 94
Total is 4096


In [110]:
df_

,watch_list,test_in,test_out,recommendation
26208,"[2595, 3021, 3023, 3020, 2600, 2596, 609, 2587...","[3021, 3023, 3020, 2600, 2596, 609, 2599, 782,...","[2587, 2595]","[731, 3515, 739, 4257, 4033, 4279, 2602, 778, ..."
26211,"[710, 2599, 734, 644, 646, 2600, 634, 3569, 30...","[710, 2599, 734, 644, 646, 634, 3569, 3023, 70...","[2600, 3022]","[623, 4017, 3514, 5196, 4802, 858, 3457, 720, ..."
26221,"[654, 3021, 2599, 2981, 3000, 2990, 2602, 705,...","[654, 3021, 2599, 2981, 3000, 2990, 2602, 705,...","[3020, 3008]","[778, 787, 731, 3515, 4017, 3514, 647, 5984, 6..."
26238,"[1179, 627, 735, 707, 3021, 653, 696, 3567, 30...","[1179, 627, 735, 707, 3021, 653, 696, 3567, 30...","[3023, 3539]","[629, 714, 740, 739, 3121, 731, 3515, 778, 350..."
26269,"[621, 2979, 3023, 3020, 3021, 2595, 2596, 3009...","[621, 2979, 3023, 3020, 3021, 2596, 3009, 770,...","[3007, 2595]","[718, 4782, 5286, 739, 4257, 4033, 731, 3515, ..."
...,...,...,...,...
83061,"[707, 1706, 677, 2979, 4092, 3386, 4093, 3502,...","[707, 1706, 677, 2979, 4092, 3386, 4093, 3502,...","[4094, 2600]","[3121, 3493, 4773, 785, 679, 4782, 5286, 3501,..."
83432,"[3644, 3355, 3646, 3354, 3645, 693, 3356]","[3644, 3646, 3354, 693, 3356]","[3355, 3645]","[3582, 6398, 3692, 3612, 3501, 5962, 3843, 647..."
83451,"[3356, 648, 3645, 3354, 620, 3644, 3139, 3646,...","[3356, 648, 3645, 3354, 620, 3644, 3646, 676, ...","[3139, 3355]","[647, 5984, 3371, 4607, 3271, 3269, 3501, 5962..."
83887,"[626, 643, 612, 655, 676, 682, 3701, 3700]","[626, 643, 612, 676, 682, 3701]","[3700, 655]","[3369, 4799, 785, 4042, 711, 3983, 3455, 6383,..."
